In [2]:
!pip install llama-index

In [3]:
pip install --upgrade llama-index


In [4]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [5]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.8 MB/s eta 0:00:00


In [6]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

#OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieving documents from the index
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combining the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
    input_variables=["transcript", "knowledge_base"],
    template=(
        "The following is a clinical conversation between a doctor and a patient. "
        "Based on this conversation transcript and using relevant information from the provided knowledgebase, "
        "generate a detailed History of Present Illness (HPI) for the patient. The HPI should include the following sections:\n\n"
        "1. Chief Complaint: Brief description of only the primary symptom.Dont mention the age of the patient.\n"
        "2. Previous Occurrences: Any similar past episodes.\n"
        "3. Family History: Relevant family medical history.\n"
        "4. Symptoms: only current symptoms.\n\n"
        "Conversation Transcript:\n{transcript}\n\n"
        "Relevant Knowledge Base:\n{knowledge_base}\n\n"
        "Please ensure the HPI is comprehensive, clear, and formatted properly."
    )
)




        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)


transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)

<ipython-input-6-d13ec98dc1e7>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




History of Present Illness (HPI):
- Chief Complaint: Stomach pains
- Previous Occurrences: Last night, after eating Chinese food, patient experienced stomach pain which started in the middle and moved to the right. Patient also experienced stomach pain as a child when her appendix was taken out.
- Family History: Patient's mother had similar gallbladder attacks after eating specific foods.
- Symptoms: Current symptoms include stomach pain, nausea, and diarrhea.
- Onset: Symptoms started last night after eating Chinese food.
- Location: Stomach pain initially in the middle, now located in the right hypogastric and right upper quadrant area.
- Character: Pain described as a twinge at first, but has worsened throughout the day.
- Severity: Pain initially rated as a 6 out of 10, but has worsened and is now a 6 out of 10.
- Associated symptoms: Nausea and diarrhea.
- Timing: Symptoms started last night and have worsened throughout the day.
- Modifying factors: Taking Tums initially provid

In [7]:
#ROS

In [37]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top results, limiting the total length
        combined_text = ""
        for result in search_results:
            combined_text += result.text.replace("Patient", "Other Patient") + "\n"
            if len(combined_text.split()) > 3000:  # Adjust this limit as needed
                break

        return combined_text

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "Based on the provided clinical conversation and the Conversation Transcript, generate a detailed Review of Systems (RoS) for the patient based on Conversation Transcript. "
                "The RoS should cover the following body systems and include specific details if mentioned, otherwise state 'denies any issues':\n\n"
                "Write Ros based on the Conversation Transcript."

                "1. General: Chief complaint\n"
                "2. Skin: Any skin issues or rashes.\n"
                "3. Head: Any headaches or head injuries.\n"
                "4. Eyes: Vision problems or eye pain.\n"
                "5. Ears, Nose, Throat: Hearing issues, nasal congestion, sore throat, etc.\n"
                "6. Cardiovascular: Heart problems, chest pain, etc.\n"
                "7. Respiratory: Breathing issues, cough, etc.\n"
                "8. Gastrointestinal: Abdominal pain, nausea, etc.\n"
                "9. Genitourinary: Urination problems, etc.\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Knowledge Base,if only  any reference is present:\n{knowledge_base}\n\n"
                "Please ensure the RoS is comprehensive, clear, and formatted properly."
                "Please go through the entire Conversation Transcript and mention yes only if the pain is present in the Conversation Transcript."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-37-2f4d3b1b2a1b>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)



1. General: Presents with stomach pain, nausea, bloating, and diarrhea. Denies any fever or chills.
2. Skin: Denies any skin issues or rashes.
3. Head: Denies any headaches or head injuries.
4. Eyes: Denies any vision problems or eye pain.
5. Ears, Nose, Throat: Denies any hearing issues, nasal congestion, or sore throat.
6. Cardiovascular: Denies any heart problems or chest pain.
7. Respiratory: Denies any breathing issues or cough.
8. Gastrointestinal: Presents with abdominal pain, nausea, bloating, and diarrhea. Denies any changes in stool color or blood in stool.
9. Genitourinary: Denies any urinary symptoms.
10. Musculoskeletal: Presents with joint pain all over the body. No history of previous joint pain or injury mentioned.
11. Neurological: Denies any headaches, changes in vision, difficulty swallowing, weakness, slurred speech, confusion, or leg pain.
12. Integumentary: Denies any skin issues or rashes.
13. Endocrine: Denies any history of diabetes or thyroid problems.
14. Ps

In [ ]:
#NEW MA

In [61]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top results, limiting the total length
        combined_text = ""
        for result in search_results:
            combined_text += result.text.replace("Patient", "Other Patient") + "\n"
            if len(combined_text.split()) > 3000:  # Adjust this limit as needed
                break

        return combined_text

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
    input_variables=["transcript", "knowledge_base"],
    template=(
        "Based on the provided clinical conversation and the following transcript, generate a detailed Review of Systems (RoS) for the patient. "
        "The RoS should cover the following body systems and include specific details only if they are mentioned in the transcript. "
        "If a system is not mentioned, do not include it in the RoS:\n\n"
        "1. General: Chief complaint.\n"
        "2. Skin: Any skin issues or rashes.\n"
        "3. Head: Any headaches or head injuries.\n"
        "4. Eyes: Vision problems or eye pain.\n"
        "5. Ears, Nose, Throat: Hearing issues, nasal congestion, sore throat.\n"
        "6. Cardiovascular: Heart problems, chest pain.\n"
        "7. Respiratory: Breathing issues, cough.\n"
        "8. Gastrointestinal: Abdominal pain, nausea.\n"
        "9. Genitourinary: Urination problems.\n"
        "10. Musculoskeletal: Muscle pain, joint pain, etc.\n"
        "11. Neurological: Numbness, tingling, etc.\n"
        "12. Psychiatric: Depression, anxiety, etc.\n\n"
        "Conversation Transcript:\n{transcript}\n\n"
        "Please ensure the RoS is comprehensive, clear, and formatted properly. Only include the systems that are explicitly mentioned in the transcript."
    )
)


        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-61-6332f8caab06>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




Review of Systems (RoS) for Patient:

1. General:
- Chief complaint: Stomach pains, nausea, and diarrhea.

2. Skin:
- No mention of skin issues or rashes.

3. Head:
- No mention of headaches or head injuries.

4. Eyes:
- No mention of vision problems or eye pain.

5. Ears, Nose, Throat:
- No mention of hearing issues.
- Nasal congestion: No.
- Sore throat: No.

6. Cardiovascular:
- No mention of heart problems.
- Chest pain: No.

7. Respiratory:
- No mention of breathing issues.
- Cough: No.

8. Gastrointestinal:
- Abdominal pain: Yes, in the middle and then moved to the right.
- Nausea: Yes, but no vomiting.
- Diarrhea: Yes, but more like loose stool.
- Changes in stool color: Lighter than normal.
- Bloating: Yes, a little.
- Urinary symptoms: No.

9. Genitourinary:
- Urination problems: No.

10. Musculoskeletal:
- No mention of muscle or joint pain.

11. Neurological:
- No mention of numbness or tingling.




In [ ]:
#PE

In [27]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# TestNameExtractor class for extracting test names
class TestNameExtractor(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieving documents from the index
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combining the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
    input_variables=["transcript", "knowledge_base"],
    template=(
        "The following is a clinical conversation between a doctor and a patient. "
        "Based on this conversation transcript and use only if relevant information from the provided knowledgebase, "
        "extract only the names of the tests mentioned by the doctor during the examination present in the Conversation Transcript. "
        "Do not include any medications or treatments, only list the tests.\n\n"
        "Conversation Transcript:\n{transcript}\n\n"
        "Knowledge Base,if only  any reference is present:\n{knowledge_base}\n\n"
        "Please list only the test names mentioned by the doctor in the Conversation Transcript , excluding any medications or treatments."
    )
)





        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = TestNameExtractor(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-27-32430ca57b98>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




1. CBC (Complete Blood Count)
2. Basic Metabolic Panel
3. Liver Function Test
4. Urinalysis
5. EKG (Electrocardiogram)
6. Chest X-ray
7. Troponin
8. CT Scan (Computed Tomography)
9. Pregnancy Test


In [21]:
#Assessment Plan


In [121]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# AssessmentPlan class for generating the assessment plan
class AssessmentPlan(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieving documents from the index
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combining the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
    input_variables=["transcript"],
    template=(
        "Based on the provided clinical conversation between the doctor and the patient, generate a detailed Assessment Plan. "
        "The Assessment Plan should include only the doctor's assessment and plan, clearly separated into two sections: Assessment and Plan. "
        "Each section should be detailed and follow the structure provided in the conversation.\n\n"
        "Conversation Transcript:\n{transcript}\n\n"
        "Assessment:\n"
        "Provide a detailed assessment of the patient's condition based on the Conversation Transcript.\n\n"
        "Plan:\n"
        "1. List only the results of diagnostic tests given to the patient mentioned in Transcript Conversation.\n"
        "2. Include any relevant diagnostic results, such as EKG findings, if mentioned in the Transcript Conversation.\n"
        "3. Mention the diagnosis and follow-up instructions provided by the doctor in Transcript Conversation.\n"
        "Ensure not to include any medications or treatments that are not mentioned explicitly in the transcript."
        "Please ensure the assessment plan is comprehensive, clear, and formatted properly."
    )
)

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = AssessmentPlan(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)





<ipython-input-121-6d27c6c21585>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




Assessment:

The patient presents with complaints of stomach pain that started after consuming fried Chinese food. The pain started in the center of the abdomen and has now moved to the right side. The patient also reports feeling nauseous and having some diarrhea. On a scale of 1 to 10, the patient rates the pain as a 6. The doctor notes a history of the patient having had her appendix removed as a child. The patient also reports not having any issues with fried food in the past.

Plan:

1. Diagnostic Tests:
- CBC (Complete Blood Count)
- Basic Metabolic Panel
- Liver Function Test
- Urinalysis
- EKG (Electrocardiogram)
- Chest X-ray
- Troponin
- CT Scan of the abdomen and pelvis
- Pregnancy test
- IV fluids (1 L of normal saline)

2. Diagnostic Results:
- EKG shows minor nonspecific ST segment changes in the lateral leads and J point elevation consistent with early repolarization.
- Trace pitting edema noted on examination.

3. Diagnosis:
Based on the patient's symptoms and physica